# 기본 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


# 데이터 전처리 클래스

In [57]:
def no_space(text):
    text1 = re.sub('&nbsp;|&nbsp;|\n|\t|\r', '', text)
    text2 = re.sub('\n\n','', text1)
    return text2

def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

def load_stopwords(basepath):
    print("불용어셋을 가져오고 있습니다.")
    with open(basepath+'stopwords.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")


class Review_keyword:
    def __init__(self, minimum_count:int)->None:
        self.basepath = input("데이터를 저장하고 불러올 기본 경로를 입력해주세요")
        self.minimum_count = minimum_count
        self.word_list =  None
        self.name = None
        self.url = "hi"
        self.data = None
        self.stopwords = load_stopwords(self.basepath)
        
    def search(self) :
        self.name= input("어떤 영화를 검색하시겠습니까? ")
        url = f'https://movie.naver.com/movie/search/result.naver?query={self.name}&section=all&ie=utf8'        
        res = requests.get(url)
        index = 1
        user_dic = {}
        if res.status_code == 200:
            soup=BeautifulSoup(res.text,'lxml')
            for href in soup.find("ul", class_="search_list_1").find_all("li"): 
                print(f"=============={index}번 영화===============")
                print(href.dl.text[:-2])
                user_dic[index] = int(href.dl.dt.a['href'][30:])
                index = index+1
        movie_num = int(input("몇 번 영화를 선택하시겠습니까? (숫자만 입력)  : "))
        code = user_dic[movie_num]
        base_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page='
        self.url = base_url+'{}'
        
    def save_and_load(self,dataframe):
        dataframe.to_csv(self.basepath+f'{self.name}review.csv' , index= False)
        df = pd.read_csv(self.basepath+f'{self.name}review.csv')
        df = df.dropna()
        df = df.drop_duplicates()
        df = df.reset_index(drop=True)
        return df
            
    def crawl_review(self):
        res = requests.get(self.url)
        if res.status_code == 200:
            soup=BeautifulSoup(res.text,'lxml')
            total = soup.select('div.score_total > strong > em')[0].text
            pages = int(total.replace(',','')[:-1]) #17,921 > 17921로 변환 후 캐스팅
            print()
            print(f"{pages}개의 페이지에서 {self.name} 영화 리뷰를 모으고 있습니다.")
            time.sleep(1)
        comments = []
        stars = []
        for page in tqdm(range(1,pages+1)):
            url = self.url.format(page)
            res = requests.get(url)
            if res.status_code == 200:
                soup=BeautifulSoup(res.text,'lxml')
                star =  soup.select('div.score_result > ul > li > div.star_score > em')
                tds = soup.select('div.score_result > ul > li > div.score_reple > p > span')
                for st in star:
                    stars.append(int(st.text))
                for cmt in tds:
                    if cmt.text != '관람객' and cmt.text !='스포일러가 포함된 감상평입니다. 감상평 보기':
                        comments.append(no_space(cmt.text))
                if(len(comments) != len(stars)):
                    print(url)
                    break 
        assert len(comments) == len(stars)
        self.data = self.save_and_load(pd.DataFrame({"Review":comments, "Rank":stars}))
       
    def make_wordlist(self,reviews): #reviews = " ".join(data['Review'].tolist())
        print("리뷰들을 모아 분석하는 중입니다.....")
        #정규표현식 적용
        print("데이터 정제 중....")
        words = extract_word(reviews)
        #형태소 추출
        print("형태소 추출 중....")
        okt = Okt()
        words = okt.morphs(words,stem=True)
        #한글자 제거
        print("한글자 제거 중....")
        words = [x for x in words if len(x)>1 or x =='닉']
        #불용어 제거
        print("불용어 제거 중....")
        words = [x for x in words if x not in self.stopwords]
        #최소횟수 미만 제거
        print("의미있는 단어리스트 생성 중....")
        time.sleep(1)
        minimum_count = 3
        final = []
        for i in tqdm(range(len(words))):
            tmp = words[i]
            if words.count(tmp) >= minimum_count:
                final.append(tmp)
        self.word_list = set(final) #조건을 만족하는 단어 리스트

    def preprocess(self,text):
        text = extract_word(text)
        okt = Okt()
        text = okt.morphs(text, stem = True)
        text = [x for x in text if x in self.word_list]
        return " ".join(text)
    
    def ready(self):
        self.search()
        self.crawl_review()
        self.make_wordlist(" ".join(self.data['Review'].tolist()))
        time.sleep(0.5)
        print("작업을 완료했습니다.")

In [59]:
mc = Review_keyword(3)
mc.ready()

데이터를 저장하고 불러올 기본 경로를 입력해주세요data/
불용어셋을 가져오고 있습니다.
어떤 영화를 검색하시겠습니까? 캣츠
==============1번 영화===============

캣츠 (Cats)

4.60 (참여 5531명)

뮤지컬, 드라마| 미국, 영국|109분 |2019
감독 : 톰 후퍼|출연 : 제니퍼 허드슨, 테일러 스위프트, 이드리스 엘바, 주디 덴치, 이안 맥켈런, 제이슨 데룰로, 제임스 코든, 레이 윈스턴, 레벨 윌슨, 프란체스카 헤이워드


==============2번 영화===============

캣츠 (Cats)
애니메이션, 코미디, 드라마, 가족| 캐나다|2분 |2014
감독 : 제프 치바 스턴즈

==============3번 영화===============

캣츠 (The Cat's Meow)
드라마, 스릴러, 코미디| 캐나다, 독일, 영국|110분 |2001
감독 : 피터 보그다노비치|출연 : 커스틴 던스트, 캐리 엘위스, 에드워드 허먼, 에디 이자드, 조안나 럼리, 빅터 슬레작, 제니퍼 틸리

==============4번 영화===============

캣츠 (The Cat)
SF, 액션, 판타지| 홍콩|88분 |1992
출연 : 글로리아 입, 이자웅, 오영미

==============5번 영화===============

캣츠 앤 독스 2 (Cats & Dogs: The Revenge Of Kitty Galore)

7.58 (참여 248명)

액션, 코미디, 가족| 미국, 오스트레일리아|82분 |2010
감독 : 브래드 페이튼|출연 : 크리스 오도넬, 베트 미들러, 닉 놀테, 제임스 마스던, 크리스티나 애플게이트, 캣 윌리엄스, 닐 패트릭 해리스, 션 헤이즈, 월리스 쇼운, 로저 무어, 조 판톨리아노, 마이클 클락 던칸


몇 번 영화를 선택하시겠습니까? (숫자만 입력)  : 5

24개의 페이지에서 캣츠 영화 리뷰를 모으고 있습니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:03<00:00,  7.67it/s]


리뷰들을 모아 분석하는 중입니다.....
데이터 정제 중....
형태소 추출 중....
한글자 제거 중....
불용어 제거 중....
의미있는 단어리스트 생성 중....


100%|█████████████████████████████████████████████████████████████████████████████| 723/723 [00:00<00:00, 45292.69it/s]


작업을 완료했습니다.


In [61]:
mc.name

'캣츠'

In [60]:
mc.data

,Review,Rank
0,3편나와라~~,10
1,귀여우니까 용서가되요,10
2,1편만큼 재밌는데ㅋㅋ비둘기개웃김ㅋㅋㅋㅋ 걍 가볍게보기좋음,10
3,재밌쪙 3편 빨뤼 만들어줘용,10
4,재밌어요,10
...,...,...
232,잡종!!,1
233,우리아이도 보고 실망한영화....,1
234,아 평점이 이럴수가......,1
235,꼬마애들도 전혀 웃지 않던 난감한영화..ㅡ.ㅡ;,2


In [55]:
mc.preprocess(mc.data['Review'][35])

'때우다 좋다'

In [56]:
mc.word_list

{'가족',
 '가족영화',
 '강아지',
 '강추',
 '고양이',
 '괜찮다',
 '귀엽다',
 '나름',
 '나와라',
 '날리다',
 '내용',
 '대박',
 '더빙',
 '동물',
 '들이다',
 '딕스',
 '때우다',
 '만들다',
 '밉다',
 '별로',
 '보기',
 '비둘기',
 '빵터지다',
 '생각',
 '속편',
 '수준',
 '스토리',
 '시사회',
 '아깝다',
 '아들',
 '어른',
 '오랜',
 '웃기다',
 '유치하다',
 '유쾌하다',
 '자다',
 '자막',
 '장면',
 '재미',
 '재미없다',
 '재미있다',
 '재밌다',
 '전작',
 '전편',
 '조카',
 '좋다',
 '좋아하다',
 '중간',
 '지루하다',
 '차다',
 '처음',
 '최고',
 '추석',
 '캣츠',
 '키티',
 '패러디',
 '평점',
 '표정'}